In [1]:
import json
import os
import pandas as pd

transcript_dir = '../../data/transcripts'
annotations = json.load(open('../../data/annotation.json'))
video_lengths = {date: x[-1]['video_end'] for date, x in annotations.items()}

In [8]:

def get_transcript(idx, window_radius, v_length, transcripts):
    frame = {}
    start = idx / 30 - window_radius
    end = idx / 30 + window_radius
    start = max(start, 0.0)
    end = min(end, v_length)
    transcript = transcripts[(transcripts['Global start'] < end) * (transcripts['Global end'] > start)]
    transcript = ' '.join(transcript['Sentence'].tolist())
    frame['video_start'] = start
    frame['video_end'] = end
    frame['transcript'] = transcript
    return frame

def construct(split, window_length):
    '''
        Construct the start and end times and corresponding transcripts for split `split` with w = `window_length` seconds
    '''
    window_radius = window_length / 2
    frames_fn = f'{split}.json'
    frames = json.load(open(frames_fn))

    data = {}
    for date in frames:
        print('Processing:', date)
        transcripts = pd.read_csv(f'{transcript_dir}/{date}.csv', encoding='cp1252')
        data[date] = []
        v_length = video_lengths[date]

        for i, frame in enumerate(frames[date]):
            q = {'query_frame_index': frame['query_frame_index']}
            q.update(get_transcript(frame['query_frame_index'], window_radius, v_length, transcripts))
            c1 = {'frame_index': frame['candidate_frame_indices'][0]}
            c1.update(get_transcript(frame['candidate_frame_indices'][0], window_radius, v_length, transcripts))
            c2 = {'frame_index': frame['candidate_frame_indices'][1]}
            c2.update(get_transcript(frame['candidate_frame_indices'][1], window_radius, v_length, transcripts))
            q.update({
                'candidates': [c1, c2],
                'label': frame['label'],
                'id': frame['id']
            })
            data[date].append(q)
    os.makedirs('./context', exist_ok=True)
    with open(f'context/RT_w{window_length}s_{split}.json', 'w') as f:
        f.write(json.dumps(data, indent = 4))

In [9]:
# Construct context windows for the validation set with w = 20s
construct(split = 'val', window_length = 20)

Processing: 03152022


Processing: 11152022
